# Homework 7 - Understand what Generative AI is thinking

In [2]:
!pip install inseq==0.5.0
!pip install transformers
!pip install bitsandbytes
!pip install accelerate

  Using cached inseq-0.5.0-py3-none-any.whl (136 kB)
  Using cached captum-0.7.0-py3-none-any.whl (1.3 MB)
  Using cached jaxtyping-0.2.28-py3-none-any.whl (40 kB)
  Using cached nvidia_cublas_cu11-11.11.3.6-py3-none-manylinux1_x86_64.whl (417.9 MB)
  Using cached nvidia_cuda_cupti_cu11-11.8.87-py3-none-manylinux1_x86_64.whl (13.1 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (23.2 MB)
  Using cached nvidia_cuda_runtime_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (875 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.6 MB/s eta 0:00:

## Machine Translation Task

In [3]:
import inseq

# List of attribution methods to be used
attribution_methods = ['saliency', 'attention']

for method in attribution_methods:
    print(f"======= Attribution Method: {method} =======")
    # Load the Chinese-to-English translation model and set up the attribution method
    model = inseq.load_model("Helsinki-NLP/opus-mt-zh-en", method)

    # Apply attribution to the input text using the specified method
    attribution_result = model.attribute(
        input_texts="我喜歡機器學習和人工智慧。",
        step_scores=["probability"],
    )

    # Remove '▁' from the tokenizer in the prefix to avoid confusion (You can ignore this part of code)
    for attr in attribution_result.sequence_attributions:
        for item in attr.source:
            item.token = item.token.replace('▁', '')
        for item in attr.target:
            item.token = item.token.replace('▁', '')

    # Display the attribution results
    attribution_result.show()

======= Attribution Method: saliency =======


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


...

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Attributing with saliency...: 100%|██████████| 10/10 [00:00<00:00, 20.93it/s]


,I,like,machine,learning,and,artificial,intelligence,.,</s>
我喜歡,0.23,0.438,0.084,0.064,0.183,0.052,0.058,0.25,0.193
機器,0.162,0.135,0.491,0.225,0.13,0.069,0.07,0.141,0.151
學,0.074,0.054,0.151,0.188,0.07,0.03,0.032,0.069,0.061
習,0.07,0.078,0.106,0.267,0.118,0.052,0.055,0.083,0.064
和,0.067,0.047,0.026,0.05,0.158,0.036,0.03,0.073,0.073
人工,0.099,0.052,0.039,0.063,0.104,0.399,0.203,0.091,0.114
智慧,0.113,0.068,0.039,0.073,0.109,0.274,0.437,0.111,0.153
。,0.119,0.066,0.02,0.027,0.083,0.045,0.057,0.118,0.124
</s>,0.066,0.063,0.044,0.043,0.046,0.043,0.058,0.064,0.068
probability,0.767,0.589,0.425,0.671,0.856,0.647,0.866,0.891,0.896


======= Attribution Method: attention =======


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Attributing with attention...: 100%|██████████| 10/10 [00:00<00:00, 54.45it/s]


,I,like,machine,learning,and,artificial,intelligence,.,</s>
我喜歡,0.19,0.517,0.28,0.018,0.048,0.032,0.014,0.049,0.023
機器,0.084,0.037,0.233,0.208,0.038,0.036,0.024,0.028,0.015
學,0.012,0.01,0.044,0.129,0.105,0.018,0.006,0.017,0.012
習,0.021,0.015,0.066,0.155,0.109,0.024,0.009,0.024,0.007
和,0.058,0.042,0.063,0.039,0.152,0.178,0.015,0.034,0.036
人工,0.023,0.015,0.022,0.027,0.025,0.26,0.183,0.033,0.016
智慧,0.029,0.017,0.033,0.046,0.04,0.132,0.365,0.155,0.013
。,0.153,0.087,0.047,0.019,0.059,0.076,0.037,0.163,0.363
</s>,0.43,0.262,0.212,0.359,0.425,0.245,0.348,0.496,0.516
probability,0.767,0.589,0.425,0.671,0.856,0.647,0.866,0.891,0.896


## Sentence Completion Task

In [4]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2-xl", load_in_8bit=True, device_map="auto")

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
for method in attribution_methods:
    # Load the model with the specified attribution method
    inseq_model = inseq.load_model(model, method)

    # Apply attribution to the input text using the specified method
    attribution_result = inseq_model.attribute(
        input_texts="The first president of America is",
        step_scores=["probability"],
    )

    # Remove 'Ġ' from GPT2's BPE tokenizer in the prefix to avoid confusion (You can ignore this part of code)
    for attr in attribution_result.sequence_attributions:
        for item in attr.source:
            item.token = item.token.replace('Ġ', '')
        for item in attr.target:
            item.token = item.token.replace('Ġ', '')

    # Display the attribution results
    attribution_result.show()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Attributing with saliency...: 100%|██████████| 20/20 [00:09<00:00,  1.44it/s]


,a,man,named,George,Washington,.,He,was,a,man,of,great,integrity,","
The,0.192,0.186,0.187,0.163,0.137,0.256,0.146,0.114,0.152,0.119,0.093,0.088,0.11,0.083
first,0.156,0.119,0.128,0.104,0.076,0.092,0.073,0.069,0.073,0.07,0.069,0.057,0.064,0.045
president,0.257,0.236,0.196,0.196,0.163,0.134,0.137,0.127,0.122,0.12,0.094,0.114,0.117,0.099
of,0.081,0.085,0.078,0.076,0.051,0.059,0.044,0.045,0.047,0.049,0.028,0.043,0.034,0.027
America,0.227,0.182,0.209,0.227,0.128,0.112,0.105,0.098,0.107,0.112,0.061,0.075,0.085,0.068
is,0.087,0.093,0.059,0.046,0.048,0.078,0.055,0.055,0.061,0.048,0.046,0.042,0.044,0.036
a,,0.099,0.029,0.026,0.028,0.03,0.032,0.028,0.031,0.035,0.034,0.026,0.024,0.02
man,,,0.113,0.062,0.054,0.048,0.058,0.06,0.065,0.119,0.07,0.047,0.054,0.046
named,,,,0.1,0.083,0.068,0.093,0.09,0.07,0.061,0.11,0.076,0.059,0.057
George,,,,,0.232,0.049,0.084,0.083,0.067,0.067,0.057,0.07,0.066,0.058


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Attributing with attention...: 100%|██████████| 20/20 [00:03<00:00,  4.02it/s]


,a,man,named,George,Washington,.,He,was,a,man,of,great,integrity,","
The,0.687,0.669,0.668,0.639,0.67,0.619,0.51,0.546,0.54,0.561,0.543,0.573,0.587,0.544
first,0.039,0.03,0.025,0.019,0.015,0.016,0.023,0.015,0.014,0.012,0.012,0.009,0.009,0.01
president,0.06,0.05,0.039,0.039,0.032,0.03,0.028,0.025,0.025,0.021,0.019,0.015,0.015,0.016
of,0.052,0.038,0.04,0.036,0.035,0.029,0.023,0.023,0.019,0.016,0.017,0.02,0.016,0.017
America,0.066,0.047,0.038,0.038,0.028,0.026,0.025,0.022,0.021,0.017,0.015,0.015,0.013,0.015
is,0.096,0.078,0.071,0.057,0.04,0.043,0.043,0.037,0.032,0.02,0.022,0.017,0.015,0.018
a,,0.087,0.07,0.051,0.036,0.036,0.035,0.029,0.028,0.028,0.024,0.02,0.017,0.017
man,,,0.05,0.055,0.03,0.031,0.029,0.025,0.024,0.022,0.024,0.017,0.016,0.017
named,,,,0.067,0.055,0.045,0.04,0.033,0.028,0.024,0.035,0.019,0.016,0.017
George,,,,,0.059,0.059,0.034,0.024,0.022,0.022,0.017,0.017,0.014,0.013
